In [35]:
# Load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
import os
import pandas as pd
import librosa
import speech_recognition as sr
import numpy as np
from sklearn import metrics
import pickle

In [36]:
filelist = os.listdir('Sound recordings2//correct')
df_correct = pd.DataFrame(filelist)
df_correct['label']='5'
df_correct = df_correct.rename(columns={0:'file'})
df_correct

,file,label
0,bo1.wav,5
1,bo2.wav,5
2,bo3.wav,5
3,bo4.wav,5
4,bo5.wav,5
...,...,...
153,o_open_(8).wav,5
154,o_open_(9).wav,5
155,to1.wav,5
156,to2.wav,5


In [37]:
filelist = os.listdir('Sound recordings2//wrong')
df_wrong = pd.DataFrame(filelist)
df_wrong ['label']='6'
df_wrong = df_wrong .rename(columns={0:'file'})
df_wrong 

,file,label
0,bc1.wav,6
1,bc2.wav,6
2,bc3.wav,6
3,bc4.wav,6
4,bc5.wav,6
...,...,...
283,tc1.wav,6
284,tc2.wav,6
285,tc3.wav,6
286,tc4.wav,6


In [38]:
df = pd.concat([ df_correct,df_wrong ], ignore_index=True)
df.head()

,file,label
0,bo1.wav,5
1,bo2.wav,5
2,bo3.wav,5
3,bo4.wav,5
4,bo5.wav,5


In [39]:
def extract_speech_features(files,name="allSound"):
#     try:
        # Sets the name to be the path to where the file is in my computer
        file_name = os.path.join(os.path.abspath('Sound recordings2/{}').format(name)+ ('\\') +str(files['file']))

        # Loads the audio file as a floating point time series and assigns the default sample rate
        # Sample rate is set to 22050 by default
        
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        HOP_LENGTH = 512
        FRAME_SIZE = 1024
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        # Computes spectral centroied
        sc = np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
        # Computes spectral bandwidth
        spectral_bw = np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
        #Computes RMSE
        rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
        #Computes Zero-crossing rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
        #Computes rolloff
        rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
        # We add also the classes of each file as a label at the end
        label = files.label
        
        
        return mfccs,sc , spectral_bw,zcr,rolloff,  label

In [40]:
def feat(features_label):
    features = []
    for i in range(0, len(features_label)):
        features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))
    return features    

In [41]:
features_label = df.apply(extract_speech_features, axis=1)
features=feat(features_label)


<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[-0.10340722  0.05008804  0.14825112 ... -0.00309645 -0.00327584
 -0.00299492] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[-0.10340722  0.05008804  0.14825112 ... -0.00309645 -0.00327584
 -0.00299492] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.00544917  0.00336349  0.01313384 ... -0.00745156 -0.01059318
 -0.01353375] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-116

<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[0.         0.         0.         ... 0.00011472 0.00011115 0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[0.         0.         0.         ... 0.00011472 0.00011115 0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.0058521e-04
 -7.1545721e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39

<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ... -3.148623e-05 -9.212239e-05
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ... -3.148623e-05 -9.212239e-05
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -7.6989752e-05
 -1.3680765e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=H

<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 2.5762500e-05 -2.2055296e-04 -2.2674202e-04 ...  1.2861819e-04
  1.3178452e-04  7.7005265e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 2.5762500e-05 -2.2055296e-04 -2.2674202e-04 ...  1.2861819e-04
  1.3178452e-04  7.7005265e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[-3.3678449e-04 -5.9694779e-04  2.7072491e-04 ...  1.1306884e-05
  8.5047853e-09 -4.4676224e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, 

<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 3.9195973e-05  2.3236817e-04  2.8756008e-04 ...  8.1018341e-05
  1.3278377e-05 -6.6081804e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 3.9195973e-05  2.3236817e-04  2.8756008e-04 ...  8.1018341e-05
  1.3278377e-05 -6.6081804e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 1.4379167e-04  2.5773114e-05 -2.6596219e-05 ... -2.4408779e-04
 -4.2548365e-04 -6.1328511e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, 

...

<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00011397 -0.00016761
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -7.1202201e-05
 -3.2948505e-04 -3.8354180e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -7.1202201e-05
 -3.2948505e-04 -3.8354180e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)

<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[0.         0.         0.         ... 0.0001273  0.00012708 0.00011761] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[0.000000e+00 0.000000e+00 0.000000e+00 ... 8.255344e-06 5.392915e-05
 0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[0.000000e+00 0.000000e+00 0.000000e+00 ... 8.255344e-06 5.392915e-05
 0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0

<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.3726453e-05
 -1.4574684e-05 -5.0678914e-06] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.0170556e-04
 -8.2055318e-05 -5.2805248e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.0170556e-04
 -8.2055318e-05 -5.2805248e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sampl

<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.9493482e-05 7.7867306e-05
 2.6490330e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.9493482e-05 7.7867306e-05
 2.6490330e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00158008  0.00072855
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=

<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  1.1660655e-05
 -1.6756602e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39-11688fb7f0c0>:18: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00011356 -0.00011968
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = np.mean(librosa.feature.rms(X, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH).T,axis=0)
<ipython-input-39-11688fb7f0c0>:22: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00011356 -0.00011968
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = np.mean(librosa.feature.spectral_rolloff(X, sr=sample_rate ).T,axis=0)
<ipython-input-39

In [42]:
X_train, X_test, y_train, y_test = train_test_split(features,df['label'], test_size=0.3, random_state=1)

In [49]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
pickle.dump(clf,open("Classifspeech.pickle",'wb')) 

In [50]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7910447761194029
